In [1]:
#New outlier detector
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from sktime.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

In [31]:
plt.rcParams['figure.figsize'] = [14, 8]

np.random.seed(3141592)

DATA_PATH = "data"

X_train = pd.read_csv(f'{DATA_PATH}/X_train.csv', index_col='id')
y_train = pd.read_csv(f'{DATA_PATH}/y_train.csv', index_col='id')
X_test = pd.read_csv(f'{DATA_PATH}/X_test.csv', index_col='id')
data = np.concatenate([X_train, y_train], axis=1)

,y
id,
0.0,74.0
1.0,51.0
2.0,70.0
3.0,52.0
4.0,85.0
...,...
1207.0,70.0
1208.0,86.0
1209.0,68.0


In [ ]:
def remove_outliers(X, y, X_od, outlier_detector):
    pred = outlier_detector.predict(X_od)
    inliers = (pred == 1)
    return X[inliers], y[inliers]

def outlier_cnt(X, outlier_detector):
    pred = outlier_detector.predict(X)
    outlier_cnt = sum(pred == -1)
    return outlier_cnt

#export ourliers into a csv
def export_outliers(X, y, X_od, outlier_detector):
    pass

The goal is to create a different plot than previous.
That shows, 
1) For different combination of OD, which points/ how many points will be considered as outliers. 

2) Or for every points, which ODs have considered it to be a outliers, so we could make a majority voting afterwards. 
>>More doable

Second idea:
We try to create a class for each model, 


We try to create a class that receives as input a series of model's names and the corresponding parameters, then return an array that indicates, for every configuration of outlier detector(same model with different sets of parametes are consiered different configurations), which are the indices of outliers.

In [49]:
Example_input = [ {"IsolationForest":{'n_estimators': 50}},
                {"IsolationForest":{'n_estimators': 100}},
                {"IsolationForest":{'n_estimators': 200}},
                {"EllipticEnvelope":{'contamination': 0.01}},
                {"EllipticEnvelope": {'contamination': 0.1}},
                {"EllipticEnvelope": {'contamination': 0.5}},
                {"LocalOutlierFactor": {'n_neighbors': 10}},
                {"LocalOutlierFactor": {'n_neighbors': 25}},
                {"LocalOutlierFactor": {'n_neighbors': 50}} ]

In [68]:
Imputer = IterativeImputer(initial_strategy='median', n_nearest_features=5)
data_imputed = Imputer.fit_transform(X_train)

In [67]:
class ReturnOutlierPrediction:
    def __init__(self, detector_param_list) :
        models = []
        for item in detector_param_list:
            for key, value in item.items():
                model =  eval(key)(**value)
                print(key,value,model)
                models.append(model)

        self.models = models

    def fit(self, data):
        for model in self.models:
            print("fit",model)
            model.fit(data)

    def predict(self,data):
        prediction = {}
        for i, model in enumerate(self.models):
            if  type(model).__name__ == "LocalOutlierFactor": #need to set novelty param
                pred = model.fit_predict(data)
            else:
                pred = model.predict(data)
                prediction[i] = pred
        return prediction

In [69]:
OutlierPredictions = ReturnOutlierPrediction(Example_input)
OutlierPredictions.fit(data_imputed)
prediction = OutlierPredictions.predict(data_imputed)

IsolationForest {'n_estimators': 50} IsolationForest(n_estimators=50)
IsolationForest {'n_estimators': 100} IsolationForest()
IsolationForest {'n_estimators': 200} IsolationForest(n_estimators=200)
EllipticEnvelope {'contamination': 0.01} EllipticEnvelope(contamination=0.01)
EllipticEnvelope {'contamination': 0.1} EllipticEnvelope()
EllipticEnvelope {'contamination': 0.5} EllipticEnvelope(contamination=0.5)
LocalOutlierFactor {'n_neighbors': 10} LocalOutlierFactor(n_neighbors=10)
LocalOutlierFactor {'n_neighbors': 25} LocalOutlierFactor(n_neighbors=25)
LocalOutlierFactor {'n_neighbors': 50} LocalOutlierFactor(n_neighbors=50)
fit IsolationForest(n_estimators=50)
fit IsolationForest()
fit IsolationForest(n_estimators=200)
fit EllipticEnvelope(contamination=0.01)


c:\Users\zheng\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(


fit EllipticEnvelope()


c:\Users\zheng\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(


fit EllipticEnvelope(contamination=0.5)


c:\Users\zheng\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(


fit LocalOutlierFactor(n_neighbors=10)
fit LocalOutlierFactor(n_neighbors=25)
fit LocalOutlierFactor(n_neighbors=50)


1212

In [74]:
def plot_detector_votings(Detector_class, Prediction):
    detectors = Detector_class.models
    outliers_dict = {}
    for model_index, prediction in Prediction:
        for p in prediction:# for every outlier index in the prediction 
            if p not in outliers_dict.keys():
                outliers_dict[p] = []
            outliers_dict[p].append(model_index)
    #now I hava a dictionary where the key is the outliers and values is a list of models
    #that mark it as outlier
    num_detectors = len(detectors)
    result_outliers_coord = {}

    for outlier, detectors_marked in outliers_dict:
        zero_coord = [0] * num_detectors
        result_outliers_coord[outlier] = zero_coord
        for detector_index in detectors_marked:
            result_outliers_coord[outlier][detector_index] = 1
    
    return result_outliers_coord
   

    


In [75]:
res = plot_detector_votings()

TypeError: plot_detector_votings() missing 2 required positional arguments: 'Detector_class' and 'Prediction'